# Neurorobotics with PyNN and PyBullet

In this Notebook we will build and execute a neuro-robotics experiment. Für the neural simulation we will use [PyNN](https://neuralensemble.org/PyNN/) and [Nest](https://www.nest-initiative.org/?page=Software). For the simulation of the physic we use [PyBullet](https://github.com/bulletphysics/bullet3/tree/master/examples/pybullet). 

This experiment is a re-creation of the [braitenberg-husky-experiment](https://bitbucket.org/hbpneurorobotics/experiments/src/development/braitenberg_husky/) in the neurorobotics plattform of the human brain project. 

We start by importing all modules we will use in this notebook. Make sure to install only python**3**-versions of all dependencies. 

You can not just install these via pip:

- _pybullet_ needs to be installed from source to get numpy-support. Otherwise it will be slower, and getCameraImage doesn't return a numpy-array. 
- _nest_ can only be installed from source. We must use version 2.16, because the current master is not yet compatible with _pynn_. Also _nest_ is compiled with _libnreuosim_, which needs a workaround until [the PR](https://github.com/nest/nest-simulator/pull/1235) is merged. I'm not 100% certain if _libneurosim_ is even required for this project, but _nest_ gives a warning if it's missing, so we will install it. 
- there is a warning "UserWarning: Unable to install NEST extensions. Certain models may not be available", which doesn't seem to affect this project. Please ignore it. 

In [1]:
import time
import numpy as np
import csv
import random

import subprocess
import re

from matplotlib import pyplot as plt
from matplotlib import animation
from pyNN.utility.plotting import Figure, Panel
from quantities import mV

In [2]:
def isFinished(processes):
    
    for process in processes:
        if(process.poll() == None):
            return False
    
    return True

In [3]:
output_min = []
error_min = 200
#number_params = 4
number_params = 8

print("Starting")

for N in range(20):
    
    # Start time measurement
    start = time.time()
    
    weights_list = []
    processes = []
    
    # Execute all subprocesses and save outputs into files
    for i in range(40):
        
        # Generate Random weights
        weights = []
        for _ in range(number_params):
            #weights.append( str(10**random.uniform(-4, -2)) ) 
            weights.append( str(10**random.uniform(-5, 3)) ) 

        # Spawn the subprocesses and write outputs into files
        filename = "temp/log" + str(i) + ".txt"
        with open(filename, "w") as file:
            command = ['python', 'OptimizationJob_BigBrain.py']
            command.extend(weights)
            processes.append( subprocess.Popen(command, stdout=file) ) 
            
        # Append current weights to weights list
        weights_list.append(weights)
            
    # Wait until all processes are finished
    while isFinished(processes) == False:
        time.sleep(0.5)
    
    # Read outputs
    for i in range(40):

        with open("temp/log" + str(i) + ".txt") as file:
            filetext = file.read()

            # Find fitness value through regex
            matches = re.findall("(.*[^=])=(.*)", filetext)
            error = float(matches[0][1])

            # A new minimum is found
            if error < error_min:
                error_min = error
                print(weights_list[i])
                print(error)

    # Time measurement
    end = time.time()
    
    # Ausgabe
    print('Time taken in seconds -', end - start)
    
    
print("Finished")

Starting
['261.34041305725606', '0.001504883510426827', '46.23787087963139', '130.2181077025043', '0.1767556019601011', '0.7961064195468438', '0.013236660144399918', '4.57952345241388']
1.6103855407955052
['49.7135280747339', '0.0007556014974569722', '18.472215791693465', '417.4003160554351', '1.1863778339556252', '3.135825725527796e-05', '7.745272000819369', '6.738756517265792e-05']
1.5439414106831666
['1.079215322563499e-05', '0.0028126853044296166', '50.89764859487136', '8.3948403345601', '0.00034382233306911507', '0.0004168449581905281', '7.390112298087331', '0.012065612868626664']
1.5255471897720245
['0.011083929028975353', '5.725338810417925e-05', '8.67529494254232', '64.63715796799788', '0.5599740533345419', '0.035187209272501546', '126.46496613609685', '3.46980311417372e-05']
1.4496532944590168
Time taken in seconds - 252.56341528892517
['0.0067949495737044535', '0.01517198917759644', '14.022875700801363', '0.006755901493199507', '0.00019437571420171363', '28.47663566707407', '

KeyboardInterrupt: 

### Time measurement for small brain

| Number of subprocesses   |  Elapsed Time   |  Elapsed Time per subprocess  |
|--------------------------|-----------------|-------------------------------|
| 1                        |    39.06 s      |         39.06 s               |
| 5                        |    42.63 s      |          8.52 s               |
| 10                       |    44.72 s      |          4.47 s               |
| 20                       |    49.58 s      |          2.48 s               |
| 30                       |    66.34 s      |          2.21 s               |
| 40                       |    73.10 s      |          1.83 s               |
| 50                       |    92.97 s      |          1.86 s               |
| 80                       |   149.80 s      |          1.87 s               |
|100                       |   188.15 s      |          1.88 s               |
|200                       |   382.65 s      |          1.91 s               |
|300                       |   592.33 s      |          1.97 s               |
|500                       |   993.66 s      |          1.99 s               |
|800                       |  1599.73 s      |          1.99 s               |